In [1]:
import numpy as np
import tensorflow as tf
from scipy import spatial
from scipy import stats
from gensim.models import word2vec

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re
import sys
import gc

# Self define module
from mini_batch_helper import extractor
from mini_batch_helper import MiniBatchCorpus

### Loading corpus and forming dict

In [2]:
word2vec_fname = 'models/word2vec/vec512_win15_iter15_mincnt5.bin'
corpus_fnames = [
    'datas/training_data/下課花路米.txt',
    'datas/training_data/人生劇展.txt',
    'datas/training_data/公視藝文大道.txt',
    'datas/training_data/成語賽恩思.txt',
    'datas/training_data/我的這一班.txt',
    'datas/training_data/流言追追追.txt',
    'datas/training_data/聽聽看.txt',
    'datas/training_data/誰來晚餐.txt',
]
sample_rate_on_training_datas = 1
extra_words = ['<pad>']
unknown_word = None

word2id, id2word, word_p, embedding_matrix, corpus, corpus_id = extractor(word2vec_fname, corpus_fnames, sample_rate_on_training_datas, extra_words, unknown_word)

In [3]:
voc_size = embedding_matrix.shape[0]
emb_size = embedding_matrix.shape[1]
pad_word_id = word2id['<pad>']
max_seq_len = np.max([len(s) for cp in corpus_id for s in cp])

print('%20s: %d' % ('voc_size', voc_size))
print('%20s: %d' % ('emb_size', emb_size))
print('%20s: %d' % ('pad_word_id', pad_word_id))
print('%20s: %d' % ('max_seq_len', max_seq_len))

# Data split
rnd_idx = np.arange(len(corpus_id))
np.random.shuffle(rnd_idx)
corpus_id = corpus_id[rnd_idx[:len(corpus_id)//2]]
valid_corpus_num = 10

train_data_loader = MiniBatchCorpus(corpus_id[valid_corpus_num:])
valid_data_loader = MiniBatchCorpus(corpus_id[:valid_corpus_num])
print('train datas num:', train_data_loader.data_num)
print('valid datas num:', valid_data_loader.data_num)

            voc_size: 65865
            emb_size: 512
         pad_word_id: 65864
         max_seq_len: 76
train datas num: 2859792
valid datas num: 16526


### Define model

In [4]:
# Word embedding model
embeddings_W = tf.Variable(embedding_matrix)

# Input
x1 = tf.placeholder(tf.int32, [None, None])
x2 = tf.placeholder(tf.int32, [None, None])
y = tf.placeholder(tf.int32, [None])
lr = tf.placeholder(tf.float64)

# Sentence embedding
x1_mask = tf.to_double(tf.not_equal(x1, pad_word_id))
x2_mask = tf.to_double(tf.not_equal(x2, pad_word_id))
x1_len = tf.reduce_sum(x1_mask, axis=1)
x2_len = tf.reduce_sum(x2_mask, axis=1)
x1_embedded = tf.gather(embeddings_W, x1) * tf.reshape(x1_mask, [-1, tf.shape(x1)[1], 1])
x2_embedded = tf.gather(embeddings_W, x2) * tf.reshape(x2_mask, [-1, tf.shape(x2)[1], 1])
x1_center = tf.reduce_sum(x1_embedded, axis=1) / tf.reshape(tf.to_double(x1_len)+1e-6, [-1, 1])
x2_center = tf.reduce_sum(x2_embedded, axis=1) / tf.reshape(tf.to_double(x2_len)+1e-6, [-1, 1])

W = tf.Variable(tf.truncated_normal([emb_size, emb_size], stddev=0.01, dtype=tf.float64))
tf_score = tf.reduce_sum((x2_center * (x1_center @ W)), axis=1)

tf_prob = tf.sigmoid(tf_score)
tf_guess = tf.cast(tf.greater_equal(tf_prob, 0.5), tf.int32)
tf_correct = tf.reduce_sum(tf.cast(tf.equal(y, tf_guess), tf.int32))

In [ ]:
# # Export embedding layer for naive method to use
# saver = tf.train.Saver()
# sess = tf.Session()
# saver.restore(sess, './models/Attack-sentence-embedding-best/final')
# now_emb_w = sess.run(embeddings_W)
# with open('./models/word2vec/fine-tuned.txt', 'w') as f:
#     assert(len(id2word) == now_emb_w.shape[0])
#     f.write('%d %d\n' % (voc_size, emb_size))
#     for word, vec in zip(id2word, now_emb_w):
#         f.write('%s %s\n' % (word, ' '.join([str(f) for f in vec])))

----------------------------------
## Training

In [5]:
reg = tf.nn.l2_loss(W) * 0
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(y, tf.float64), logits=tf_score))
cost_reg = cost + reg
optimizer = tf.train.AdamOptimizer(lr)
gvs = optimizer.compute_gradients(cost_reg)
capped_gvs = [(tf.clip_by_norm(grad, 20), var) for grad, var in gvs]
train_step = optimizer.apply_gradients(capped_gvs)

In [6]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
def eval_valid_loss():
    valid_loss = 0
    valid_acc = 0
    valid_batch = 2048
    num = [0, 0]
    correct = [0, 0]
    batch_num = valid_data_loader.data_num // valid_batch
    for i in range(batch_num):
        b_x1, b_x2, b_y = valid_data_loader.next_batch(valid_batch, max_seq_len, pad_word_id)
        now_loss, now_correct, now_guess = sess.run([cost, tf_correct, tf_guess], {x1: b_x1, x2: b_x2, y: b_y})
        assert(now_correct == np.sum(now_guess == b_y))
        valid_loss += now_loss / batch_num
        valid_acc += now_correct / (batch_num * valid_batch)
        num[0] += np.sum(b_y == 0)
        num[1] += np.sum(b_y == 1)
        correct[0] += np.sum((b_y == 0) & (now_guess == b_y))
        correct[1] += np.sum((b_y == 1) & (now_guess == b_y))
    recall_0 = correct[0] / num[0] if num[0] else 0
    recall_1 = correct[1] / num[1] if num[1] else 0
    return valid_loss, valid_acc, recall_0, recall_1

In [9]:
learning_rate = 1e-3
batch_size = 256
epoch_num = 40
log_interval = 1
save_interval = 10000

last_epoch = None
train_batch_loss = 0
start_time = time.time()
best_acc = None
for i_batch in range(epoch_num * train_data_loader.data_num // batch_size):
    epoch = i_batch // (train_data_loader.data_num // batch_size)
    if last_epoch is None or last_epoch != epoch:
        last_epoch = epoch
        print('Start epoch %d' % (epoch))

    epoch = i_batch // (train_data_loader.data_num // batch_size)
    b_x1, b_x2, b_y = train_data_loader.next_batch(batch_size, max_seq_len, pad_word_id)
    _, now_loss = sess.run([train_step, cost], {x1: b_x1, x2: b_x2, y: b_y, lr: learning_rate})
    train_batch_loss += now_loss / log_interval
    if (i_batch+1) % log_interval == 0:
        valid_loss, valid_acc, recall_0, recall_1 = eval_valid_loss()
        print('train batch loss %8f / valid loss %8f / valid acc %8f / recall_0 %8f / recall_1 %8f / elapsed time %.f' % (
            train_batch_loss, valid_loss, valid_acc, recall_0, recall_1, time.time()-start_time), flush=True)
        train_batch_loss = 0
        if best_acc is None or best_acc < valid_acc:
            best_acc = valid_acc
            print('model saved (best)', flush=True)
            saver.save(sess, 'models/Attack-sentence-embedding-512/best')
        else:
            learning_rate /= 1.01
            print('Decay learing rate -> %10f' % (learning_rate))
    if save_interval is not None and (i_batch+1) % save_interval == 0:
        saver.save(sess, 'models/Attack-sentence-embedding-512/latest')
        print('model saved (latest)', flush=True)

saver.save(sess, 'models/Attack-sentence-embedding-512/final')

Start epoch 0
train batch loss 0.956859 / valid loss 0.938181 / valid acc 0.584900 / recall_0 0.462712 / recall_1 0.707118 / elapsed time 42
model saved (best)
train batch loss 0.980248 / valid loss 0.936255 / valid acc 0.584106 / recall_0 0.597656 / recall_1 0.570557 / elapsed time 90
Decay learing rate ->   0.000990


KeyboardInterrupt: 

------------------------------
## Evaluating

In [ ]:
eval_max_seq_len = 100

In [ ]:
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

In [ ]:
sample = pd.read_csv('datas/sample_test_data.txt')

sample_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
sample_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]
sample_y = sample.answer.values
assert(np.sum([len(_)!=6 for _ in sample_x2]) == 0)

sample_x1 = [[word for word in jieba.cut(' '.join(s)) if word != ' '] for s in sample_x1]
sample_x2 = [[[word for word in jieba.cut(r) if word != ' '] for r in rs] for rs in sample_x2]

test_datas = pd.read_csv('datas/AIFirstProblem.txt')

test_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.dialogue.values]
test_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.options.values]
assert(np.sum([len(_)!=6 for _ in test_x2]) == 0)

test_x1 = [[word for word in jieba.cut(' '.join(s)) if word != ' '] for s in test_x1]
test_x2 = [[[word for word in jieba.cut(r) if word != ' '] for r in rs] for rs in test_x2]

In [ ]:
def word_lst_2_id_lst(lst):
    return [word2id[lst[i]] if i<len(lst) and lst[i] in word2id else pad_word_id for i in range(eval_max_seq_len)]

In [ ]:
sample_id1 = np.array([word_lst_2_id_lst(s) for s in sample_x1])
sample_id2 = np.array([[word_lst_2_id_lst(r) for r in rs] for rs in sample_x2])
test_id1 = np.array([word_lst_2_id_lst(s) for s in test_x1])
test_id2 = np.array([[word_lst_2_id_lst(r) for r in rs] for rs in test_x2])

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, 'models/Attack-sentence-embedding-previous/final')

In [ ]:
# Generate sample answer
probs = sess.run(tf_prob, {
    x1: np.repeat(sample_id1, 6, axis=0),
    x2: sample_id2.reshape(-1, eval_max_seq_len),
})
probs = probs.reshape(-1, 6)
probs_ans = np.argmax(probs, axis=1)
np.sum(probs_ans == sample_y)

In [ ]:
# Generate test answer
probs = sess.run(tf_prob, {
    x1: np.repeat(test_id1, 6, axis=0),
    x2: test_id2.reshape(-1, eval_max_seq_len),
})
probs = probs.reshape(-1, 6)
my_test_ans = np.argmax(probs, axis=1)

In [ ]:
# with open('answer/attack-sentence-embedding.txt', 'w') as f:
#     f.write('id,ans\n')
#     f.write('\n'.join(['%d,%d' % (i+1, a) for i, a in enumerate(my_test_ans)]))
#     f.write('\n')